In [1]:
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_laobai.csv')
data_rfe=np.array(data_rfe)
fea=data_rfe[:,:49]
target=data_rfe[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

# SVM-RFE_RELIEF

In [2]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,alpha,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            #print(rfe_score)
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            if len(candi_fea)==optim_featur_num:
                return candi_fea

In [3]:
rfe_relief_model=rfe_relief_SVM(x_train,y_train,0.1)

In [4]:
aver_mean_lis1,F_measure_lis1=rfe_relief_model.featu_score(0.15,5)
print(aver_mean_lis1,F_measure_lis1)

[0.7999425857299441, 0.7917754058144998, 0.8101479722323711, 0.8075943420846601, 0.8111696852654106, 0.8101479722323713, 0.8188266610992223, 0.8188305757085443, 0.8213763766376114, 0.8223967848008769, 0.8229082937522835, 0.8223993945404249, 0.8275053499660734, 0.8269951458844407, 0.8264836369330342, 0.8224020042799728, 0.8213815961167075, 0.8198522887415836, 0.8234224124432382, 0.819338170050629, 0.8280129443081581, 0.8300498460253667, 0.8290307427318753, 0.8274988256172033, 0.8300563703742366, 0.8264771125841641, 0.8274988256172033, 0.8259682133723054, 0.8269886215355708, 0.8239260921760009, 0.8213763766376114, 0.8218891904587922, 0.8218878855890182, 0.8244402108669554, 0.820871392035075, 0.8208674774257527, 0.8254658385093168, 0.8239339213946447, 0.8203572733441202, 0.8203585782138942, 0.8198483741322617, 0.8208687822955267, 0.8229095986220576, 0.8244402108669554, 0.8213789863771594, 0.8223967848008769, 0.8183151521478156, 0.8203572733441202, 0.820870087165301] [0.8254945179243712, 0

In [5]:
aver_mean_lis2,F_measure_lis2=rfe_relief_model.featu_score(0.25,5)
print(aver_mean_lis2,F_measure_lis2)

[0.7999425857299441, 0.7917754058144998, 0.8101479722323711, 0.8075943420846601, 0.8111696852654106, 0.8101479722323713, 0.8188266610992223, 0.8188305757085443, 0.8213763766376114, 0.8223967848008769, 0.8229082937522835, 0.8234198027036902, 0.8275053499660734, 0.8269951458844407, 0.8264836369330342, 0.8224020042799728, 0.8213815961167075, 0.8198522887415836, 0.8234224124432382, 0.819338170050629, 0.8229109034918315, 0.8264823320632602, 0.8269912312751189, 0.8275027402265254, 0.8249465003392661, 0.8239260921760007, 0.8239287019155487, 0.8259682133723054, 0.8254580092906728, 0.8264784174539381, 0.8259708231118534, 0.8264823320632602, 0.8275001304869776, 0.8264823320632602, 0.8249504149485881, 0.8259708231118534, 0.8254658385093168, 0.8218878855890182, 0.8213776815073857, 0.8218878855890182, 0.8193368651808548, 0.8208687822955267, 0.8229109034918315, 0.8208661725559789, 0.8213789863771594, 0.8223967848008769, 0.8183151521478156, 0.8224006994101989, 0.820870087165301] [0.8254945179243712, 

In [6]:
aver_mean_lis3,F_measure_lis3=rfe_relief_model.featu_score(0.35,5)
print(aver_mean_lis3,F_measure_lis3)

[0.7999425857299441, 0.7917754058144998, 0.8101479722323711, 0.8075943420846601, 0.8111696852654106, 0.8101479722323713, 0.8188266610992223, 0.8188305757085443, 0.8213763766376114, 0.8223967848008769, 0.8229082937522835, 0.8223993945404249, 0.8213802912469335, 0.8259734328514015, 0.8249530246881361, 0.8224020042799728, 0.8285283678688866, 0.826486246672582, 0.820872696904849, 0.8224006994101989, 0.8249530246881361, 0.8229069888825095, 0.8218891904587922, 0.8264797223237121, 0.8234184978339162, 0.8239260921760007, 0.8229056840127354, 0.8198418497833917, 0.8213750717678374, 0.8218878855890182, 0.8239300067853229, 0.8218865807192441, 0.8203559684743462, 0.8198457643927135, 0.8218865807192444, 0.8172947439845505, 0.819335560311081, 0.819338170050629, 0.8239287019155489, 0.8223993945404248, 0.819335560311081, 0.8213763766376114, 0.8203572733441202, 0.8218891904587922, 0.8218904953285662, 0.8208674774257527, 0.8198470692624877, 0.8224006994101989, 0.820870087165301] [0.8254945179243712, 0.81

In [7]:
aver_mean_lis4,F_measure_lis4=rfe_relief_model.featu_score(0.45,5)
print(aver_mean_lis4,F_measure_lis4)

[0.7999425857299441, 0.7917754058144998, 0.8101479722323711, 0.8075943420846601, 0.8111696852654106, 0.8101479722323713, 0.8188266610992223, 0.8188305757085443, 0.8213763766376114, 0.8223967848008769, 0.8229082937522835, 0.8223993945404249, 0.8213802912469335, 0.8259734328514015, 0.8249530246881361, 0.8224020042799728, 0.8285283678688866, 0.8259773474607235, 0.823425022182786, 0.8218878855890182, 0.8218891904587922, 0.8229095986220575, 0.8223993945404249, 0.8244349913878596, 0.8269925361448929, 0.8264810271934862, 0.8229056840127354, 0.8239260921760009, 0.8244389059971816, 0.8218865807192441, 0.8244402108669554, 0.8223993945404249, 0.8203559684743462, 0.8198457643927135, 0.8218865807192444, 0.8172947439845505, 0.819335560311081, 0.819338170050629, 0.8218891904587922, 0.8229082937522836, 0.8193368651808548, 0.8203572733441202, 0.8213802912469335, 0.8218904953285662, 0.8218904953285662, 0.8208674774257527, 0.8198470692624877, 0.8224006994101989, 0.820870087165301] [0.8254945179243712, 0.

In [8]:
aver_mean_lis5,F_measure_lis5=rfe_relief_model.featu_score(0.55,5)
print(aver_mean_lis5,F_measure_lis5)

[0.7999425857299441, 0.7917754058144998, 0.8101479722323711, 0.8075943420846601, 0.8111696852654106, 0.8137207056735738, 0.8188266610992223, 0.8188305757085443, 0.8213763766376114, 0.8223967848008769, 0.8183164570175897, 0.8234198027036902, 0.8244415157367294, 0.8259734328514015, 0.8249530246881361, 0.8269938410146667, 0.8285283678688866, 0.8269977556239887, 0.8259760425909495, 0.8229148181011535, 0.8208674774257527, 0.8213789863771594, 0.8208674774257527, 0.8198483741322617, 0.8229109034918316, 0.8259708231118534, 0.8280129443081581, 0.8239260921760009, 0.8234171929641422, 0.8249504149485881, 0.8244389059971816, 0.8213776815073857, 0.8203559684743462, 0.8198457643927135, 0.8218865807192444, 0.8172947439845505, 0.8162743358212851, 0.8198457643927135, 0.8203572733441202, 0.8203559684743462, 0.8239273970457749, 0.8249491100788141, 0.8249504149485881, 0.8229095986220575, 0.8203585782138942, 0.8208674774257527, 0.8198470692624877, 0.8203572733441202, 0.820870087165301] [0.8254945179243712,

In [9]:
aver_mean_lis6,F_measure_lis6=rfe_relief_model.featu_score(0.65,5)
print(aver_mean_lis6,F_measure_lis6)

[0.7999425857299441, 0.7917754058144998, 0.8101479722323711, 0.8075943420846601, 0.8111696852654106, 0.8137207056735738, 0.8188266610992223, 0.817806252935957, 0.8188292708387703, 0.8208648676862049, 0.8183164570175897, 0.8234198027036902, 0.8244415157367294, 0.826486246672582, 0.8280142491779321, 0.8269951458844407, 0.8249543295579101, 0.8244454303460514, 0.8249530246881361, 0.8239313116550969, 0.820868782295527, 0.820871392035075, 0.8229135132313795, 0.8229109034918315, 0.8249543295579101, 0.8249530246881361, 0.8229095986220575, 0.8249491100788141, 0.8254606190302208, 0.8264797223237121, 0.8254593141604467, 0.8229082937522836, 0.8203572733441202, 0.8223980896706508, 0.8218878855890182, 0.8249504149485881, 0.8162743358212851, 0.8198457643927135, 0.8259734328514015, 0.8239313116550969, 0.8239313116550969, 0.8218891904587922, 0.820868782295527, 0.820868782295527, 0.8213776815073854, 0.8198431546531657, 0.8198470692624877, 0.8203572733441202, 0.820870087165301] [0.8254945179243712, 0.819

In [10]:
aver_mean_lis7,F_measure_lis7=rfe_relief_model.featu_score(0.75,5)
print(aver_mean_lis7,F_measure_lis7)

[0.7999425857299441, 0.7917754058144998, 0.8101479722323711, 0.8075943420846601, 0.8111696852654106, 0.8137207056735738, 0.8188266610992223, 0.817806252935957, 0.8188292708387703, 0.8208648676862049, 0.8183164570175897, 0.8178062529359569, 0.8208661725559789, 0.8244428206065034, 0.8275040450962994, 0.8269925361448929, 0.8259734328514015, 0.8229095986220576, 0.8213789863771594, 0.8213789863771594, 0.8224072237590689, 0.8218957148076622, 0.8229174278407017, 0.8218957148076622, 0.820870087165301, 0.8198470692624877, 0.819338170050629, 0.819338170050629, 0.8188227464899004, 0.8229069888825095, 0.8254593141604467, 0.8259682133723054, 0.8259695182420795, 0.8203572733441202, 0.8229109034918315, 0.8244402108669554, 0.8249504149485881, 0.8254606190302208, 0.8239313116550969, 0.8239313116550969, 0.8239313116550969, 0.8218891904587922, 0.820868782295527, 0.820868782295527, 0.8213776815073854, 0.8203533587347982, 0.8218865807192441, 0.8249517198183621, 0.820870087165301] [0.8254945179243712, 0.819

In [11]:
aver_mean_lis8,F_measure_lis8=rfe_relief_model.featu_score(0.85,5)
print(aver_mean_lis8,F_measure_lis8)

[0.7999425857299441, 0.7917754058144998, 0.8055561354976775, 0.80300381021974, 0.8040242183830054, 0.8035153191711467, 0.8045409468135081, 0.8172947439845505, 0.8157680463489744, 0.8218839709796961, 0.8229109034918315, 0.8198496790020358, 0.8224046140195208, 0.8224006994101989, 0.8203559684743464, 0.8188227464899004, 0.8188253562294484, 0.8152539276580197, 0.8203624928232163, 0.8203624928232163, 0.8218944099378882, 0.8213842058562555, 0.8198548984811316, 0.8208753066443968, 0.8172986585938723, 0.8183229813664596, 0.823425022182786, 0.8264875515423562, 0.8239313116550969, 0.8229122083616055, 0.8239313116550967, 0.8213815961167075, 0.82495432955791, 0.8264823320632602, 0.8249478052090401, 0.8244402108669554, 0.8249504149485881, 0.8254606190302208, 0.8264836369330342, 0.8269951458844407, 0.8264862466725822, 0.826486246672582, 0.820868782295527, 0.820868782295527, 0.8213776815073854, 0.8203533587347982, 0.8218865807192441, 0.8249517198183621, 0.820870087165301] [0.8254945179243712, 0.81968

In [12]:
aver_mean_lis9,F_measure_lis9=rfe_relief_model.featu_score(0.95,5)
print(aver_mean_lis9,F_measure_lis9)

[0.7999425857299441, 0.8009656036327575, 0.8009642987629834, 0.7963711571585156, 0.794330340831985, 0.7963763766376115, 0.8183164570175897, 0.8167910642517876, 0.8198535936113576, 0.8188318805783183, 0.8208740017746228, 0.8167871496424656, 0.814742418706613, 0.8132118064617151, 0.8167845399029176, 0.8188253562294484, 0.8137220105433478, 0.8127016023800826, 0.8127016023800824, 0.8132118064617151, 0.8162769455608331, 0.8157654366094264, 0.8116824990865912, 0.8142348243645283, 0.8132091967221671, 0.8157680463489744, 0.8137285348922176, 0.8172999634636463, 0.8157693512187484, 0.8234224124432382, 0.8183138472780417, 0.8208674774257527, 0.8203546636045722, 0.814741113836839, 0.8223967848008769, 0.8218852758494701, 0.819335560311081, 0.817804948066183, 0.8218852758494701, 0.8183099326687197, 0.8203507489952504, 0.8198470692624877, 0.8218878855890182, 0.8244441254762774, 0.8213776815073854, 0.8203533587347982, 0.8224006994101989, 0.8249517198183621, 0.820870087165301] [0.8254945179243712, 0.82

In [26]:
np.max(aver_mean_lis1[:20]),np.max(aver_mean_lis2[:20]),np.max(aver_mean_lis3[:20]),np.max(aver_mean_lis4[:20]),np.max(aver_mean_lis5[:20]),np.max(aver_mean_lis6[:20]),np.max(aver_mean_lis7[:20]),np.max(aver_mean_lis8[:20]),np.max(aver_mean_lis9[:20])

(0.8275053499660734,
 0.8275053499660734,
 0.8285283678688866,
 0.8285283678688866,
 0.8285283678688866,
 0.8280142491779321,
 0.8275040450962994,
 0.8229109034918315,
 0.8208740017746228)

In [27]:
np.argmax(aver_mean_lis3[:20]),np.argmax(aver_mean_lis4[:20]),np.argmax(aver_mean_lis5[:20])

(16, 16, 16)

# SVM-RFE

In [13]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,x_train[:,candi_fea],y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,x_train[:,candi_fea],y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        kf=KFold(n_splits=5,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            if len(candi_fea)==featur_num:
                break
        return candi_fea

In [14]:
rfe_SVM_model=rfe_SVM(x_train,y_train,0.1)
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis=rfe_SVM_model.featu_score(5)

In [15]:
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis

([0.7999425857299441,
  0.7917754058144998,
  0.8101479722323711,
  0.8075943420846601,
  0.8111696852654106,
  0.8101479722323713,
  0.8188266610992223,
  0.8188305757085443,
  0.8213763766376114,
  0.8223967848008769,
  0.8229082937522835,
  0.8223993945404249,
  0.8275053499660734,
  0.8269951458844407,
  0.8264836369330342,
  0.8249504149485881,
  0.8239300067853229,
  0.8259682133723054,
  0.819339474920403,
  0.820868782295527,
  0.8259747377211755,
  0.8305665744558693,
  0.8290333524714233,
  0.831075473667728,
  0.8300563703742366,
  0.8264771125841641,
  0.8274988256172033,
  0.8259682133723054,
  0.8269886215355708,
  0.8239260921760009,
  0.8213763766376114,
  0.8264823320632602,
  0.8264810271934862,
  0.8249504149485881,
  0.8269951458844407,
  0.8275066548358474,
  0.8275040450962994,
  0.8259721279816274,
  0.8229082937522836,
  0.8229095986220575,
  0.8229082937522836,
  0.8218891904587922,
  0.8218904953285662,
  0.8244428206065034,
  0.8234211075734642,
  0.819847069

# SVM-RELIEF

In [16]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        featu_num=self.X_train.shape[1]
        relief_fea_score=reliefF.reliefF(self.X_train,self.Y_train)
        relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        for i in range(featu_num):
            estimator=SVC(kernel='linear',C=self.C)
            candi_fea=relief_candi_fea[:(i+1)]
            aver=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i]=aver
            F_lis[i]=F1 
        return aver_lis,F_lis

In [17]:
relief_SVM_model=relief_SVM(x_train,y_train,0.1)
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis=relief_SVM_model.featu_score(5)

In [18]:
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis

([0.7999425857299441,
  0.7994336865180853,
  0.8055652695860953,
  0.7994415157367294,
  0.8025014353567513,
  0.794336865180855,
  0.792804948066183,
  0.7933177618873637,
  0.7917832350331437,
  0.791273030951511,
  0.7989300067853228,
  0.7968813612401482,
  0.8004540946813508,
  0.804535727334412,
  0.8035140143013727,
  0.8019834020564748,
  0.8142374341040763,
  0.814747638185709,
  0.8147489430554831,
  0.8132183308105851,
  0.8142413487133983,
  0.8172999634636463,
  0.814747638185709,
  0.8137220105433476,
  0.8188266610992223,
  0.8188292708387704,
  0.8198522887415836,
  0.817811472415053,
  0.8172934391147763,
  0.8167832350331435,
  0.817801033456861,
  0.8157615220001043,
  0.8157628268698783,
  0.816273030951511,
  0.8162756406910591,
  0.8116798893470432,
  0.814742418706613,
  0.8137233154131218,
  0.8101518868416931,
  0.8121927031682239,
  0.8137259251526698,
  0.8218891904587922,
  0.824955634427684,
  0.8244441254762774,
  0.82495432955791,
  0.8203533587347982,
 

In [19]:
laobai_accuracy=[]
laobai_accuracy.append(aver_mean_lis1)
laobai_accuracy.append(aver_mean_lis2)
laobai_accuracy.append(aver_mean_lis3)
laobai_accuracy.append(aver_mean_lis4)
laobai_accuracy.append(aver_mean_lis5)
laobai_accuracy.append(aver_mean_lis6)
laobai_accuracy.append(aver_mean_lis7)
laobai_accuracy.append(aver_mean_lis8)
laobai_accuracy.append(aver_mean_lis9)
laobai_accuracy.append(rfe_SVM_aver_mean_lis)
laobai_accuracy.append(relief_SVM_aver_mean_lis)

In [20]:
colu=[i for i in range(1,50)]
laobai_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(laobai_accuracy))
laobai_acurracy_dataframe.to_csv("laobai_accuracy.csv")

In [21]:
laobai_F_measure=[]
laobai_F_measure.append(F_measure_lis1)
laobai_F_measure.append(F_measure_lis2)
laobai_F_measure.append(F_measure_lis3)
laobai_F_measure.append(F_measure_lis4)
laobai_F_measure.append(F_measure_lis5)
laobai_F_measure.append(F_measure_lis6)
laobai_F_measure.append(F_measure_lis7)
laobai_F_measure.append(F_measure_lis8)
laobai_F_measure.append(F_measure_lis9)
laobai_F_measure.append(rfe_SVM_F_measure_lis)
laobai_F_measure.append(relief_SVM_F_measure_lis)

In [22]:
colu=[i for i in range(1,50)]
laobai_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(laobai_F_measure))
laobai_F_measure_dataframe.to_csv("laobai_F_measure.csv")